In [2]:
from docx import Document
import re

def contains_exact_keyword(text, keyword):
    # Sử dụng regex để tìm từ khóa đứng riêng biệt và chính xác về chữ hoa/thường
    pattern = r'\b' + re.escape(keyword) + r'\b'
    return bool(re.search(pattern, text))

def clean_text(text):
    # Loại bỏ phần comment bắt đầu bằng '--' hoặc '//' nhưng giữ lại dấu chấm phẩy ';'
    text = re.split(r'--|//', text)[0].strip()
    
    # Tìm dấu chấm phẩy và chỉ giữ lại phần trước dấu chấm phẩy (bao gồm cả dấu chấm phẩy)
    parts = re.split(r';', text, 1)
    if len(parts) > 1:
        return parts[0] + ';'  # Giữ lại phần trước dấu chấm phẩy và dấu chấm phẩy
    return text  # Nếu không có dấu chấm phẩy, trả về text gốc
    
def remove_text_before_keyword(text, keyword):
    # Sử dụng regex để tách chuỗi tại từ khóa và chỉ giữ lại phần sau từ khóa
    parts = re.split(r'(?i)\b' + re.escape(keyword) + r'\b', text)
    if len(parts) > 1:
        return keyword + parts[-1].strip()  # Thêm lại từ khóa vào phần sau
    return text  # Nếu từ khóa không có trong text thì trả về text gốc

def format_sql(text):
    # Thêm dấu cách giữa từ khóa SQL và phần tiếp theo nếu cần
    keywords_with_space = ['WITH', 'SELECT', 'UPDATE', 'CASE WHEN', 'CASE', 'WHERE', 'UNION ALL', 'JOIN', 'INSERT INTO', 'DELETE FROM']
    for keyword in keywords_with_space:
        text = re.sub(rf'\b{keyword}\s*', f'{keyword} ', text, flags=re.IGNORECASE)
    return text

# Hàm kiểm tra xem dòng có chứa tiếng Việt hay không
def contains_vietnamese(text):
    # Sử dụng regex để tìm các ký tự tiếng Việt
    vietnamese_pattern = re.compile(r'[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ]')
    return bool(vietnamese_pattern.search(text))

def extract_sql_code(docx_file):
    doc = Document(docx_file)
    sql_code_blocks = []
    current_block = []
    
    # Danh sách các từ khóa SQL để nhận diện bắt đầu đoạn SQL
    sql_dml_keywords = ('WITH', 'SELECT', 'UPDATE', 'INSERT INTO', 'DELETE FROM')

    # Danh sách các keyword nếu sau nó là một khoảng trắng thì ta sẽ tiếp tục lấy dòng đó
    sql_keywords = ('WITH', 'SELECT', 'UPDATE', 'CASE WHEN', 'CASE', 'WHERE', 'UNION ALL', 'JOIN', 'INSERT INTO', 'DELETE FROM')
    
    paragraphs = doc.paragraphs

    for i, para in enumerate(paragraphs):
        text = para.text.strip()
        text = clean_text(text)  # Loại bỏ phần comment nếu có
        
        # Nếu đoạn văn trống, kết thúc khối SQL hiện tại nếu cần
        if not text or para.style.name.startswith('Heading'):
            # Kiểm tra nếu đoạn trước là từ khóa và đoạn tiếp theo cũng trống thì không kết thúc
            if current_block and not any(kw in current_block[-1].upper() for kw in sql_keywords):
                sql_code_blocks.append("\n".join(current_block))
                current_block = []
            continue
        
        if any(contains_exact_keyword(text, kw) for kw in sql_dml_keywords):
            # Nếu text chứa từ khóa SQL, xóa phần đứng trước từ khóa
            for keyword in sql_dml_keywords:
                if contains_exact_keyword(text, keyword):
                    text = remove_text_before_keyword(text, keyword)
                    text = format_sql(text)
                    break
                    
            # Kiểm tra tiếng Việt sau khi đã xử lý văn bản
            if contains_vietnamese(text):
                current_block = []  # Bỏ qua khối hiện tại nếu chứa tiếng Việt
                continue
            
            current_block.append(text)
        
        # Tiếp tục thêm vào block hiện tại nếu đang trong khối SQL
        elif current_block:
            current_block.append(text)

        # Nếu đoạn văn kết thúc bằng dấu chấm phẩy, thì xem như khối SQL đã hoàn tất
        if text.endswith(';'):
        #if ';' in text:
            sql_code_blocks.append("\n".join(current_block))
            current_block = []
            continue
                
    # Trường hợp cuối cùng còn block chưa kết thúc
    if current_block:
        sql_code_blocks.append("\n".join(current_block))

    # Lọc bỏ các câu lệnh SQL trống hoặc chỉ chứa khoảng trắng
    sql_code_blocks = [sql for sql in sql_code_blocks if sql and sql.strip()]

    return sql_code_blocks

# Đọc file Word và trích xuất SQL code
sql_codes = extract_sql_code('EIB - MIS - Tài liệu Phân tích Yêu cầu - Nhóm kế hoạch - Sign NV-IT 1.2.docx')

for i, sql in enumerate(sql_codes):
    print(f"Query {i}:\n")
    print(sql)
    print("\n" + "="*80 + "\n")


In [9]:
import pandas as pd

data = []

for i, sql in enumerate(sql_codes):
    query_name = f"Query {i+1}"
    data.append([query_name, sql])

# Chuyển đổi danh sách thành DataFrame
df = pd.DataFrame(data, columns=["Query", "SQL Code"])

# Xuất DataFrame ra file Excel
df.to_excel("docx_queries.xlsx", index=False)

print("Đã xuất file Excel thành công!")

Đã xuất file Excel thành công!


In [10]:
from docx import Document
import re
import pandas as pd

def contains_exact_keyword(text, keyword):
    pattern = r'\b' + re.escape(keyword) + r'\b'
    return bool(re.search(pattern, text))

def clean_text(text):
    text = re.split(r'--|//', text)[0].strip()
    parts = re.split(r';', text, 1)
    if len(parts) > 1:
        return parts[0] + ';'
    return text
    
def remove_text_before_keyword(text, keyword):
    parts = re.split(r'(?i)\b' + re.escape(keyword) + r'\b', text)
    if len(parts) > 1:
        return keyword + parts[-1].strip()
    return text

def format_sql(text):
    keywords_with_space = ['WITH', 'SELECT', 'UPDATE', 'CASE WHEN', 'CASE', 'WHERE', 'UNION ALL', 'JOIN', 'INSERT INTO', 'DELETE FROM']
    for keyword in keywords_with_space:
        text = re.sub(rf'\b{keyword}\s*', f'{keyword} ', text, flags=re.IGNORECASE)
    return text

def contains_vietnamese(text):
    vietnamese_pattern = re.compile(r'[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ]')
    return bool(vietnamese_pattern.search(text))

def extract_sql_code(docx_file):
    doc = Document(docx_file)
    sql_data = []
    current_block = []
    current_heading = None
    
    sql_dml_keywords = ('SELECT', 'UPDATE', 'INSERT INTO', 'DELETE FROM')
    sql_keywords = ('WITH', 'SELECT', 'UPDATE', 'CASE WHEN', 'CASE', 'WHERE', 'UNION ALL', 'JOIN', 'INSERT INTO', 'DELETE FROM')
    
    paragraphs = doc.paragraphs

    for i, para in enumerate(paragraphs):
        text = para.text.strip()
        text = clean_text(text)

        # Nếu đoạn văn trống, kết thúc khối SQL hiện tại nếu cần
        if not text:
            if current_block and not any(kw in current_block[-1].upper() for kw in sql_keywords):
                sql_data.append({'Heading': current_heading, 'SQL Query': "\n".join(current_block)})
                current_block = []
            continue
        
        # Kiểm tra nếu đoạn là heading và lưu lại
        if para.style.name.startswith('Heading'):
            current_heading = para.text.strip()
            continue
        
        if any(contains_exact_keyword(text, kw) for kw in sql_dml_keywords):
            for keyword in sql_dml_keywords:
                if contains_exact_keyword(text, keyword):
                    text = remove_text_before_keyword(text, keyword)
                    text = format_sql(text)
                    break
                    
            if contains_vietnamese(text):
                current_block = []
                continue
            
            current_block.append(text)
        
        elif current_block:
            current_block.append(text)

        if text.endswith(';'):
            sql_data.append({'Heading': current_heading, 'SQL Query': "\n".join(current_block)})
            current_block = []
            continue
                
    if current_block:
        sql_data.append({'Heading': current_heading, 'SQL Query': "\n".join(current_block)})

    sql_data = [data for data in sql_data if data['SQL Query'] and data['SQL Query'].strip()]

    # Tạo DataFrame từ danh sách sql_data
    df = pd.DataFrame(sql_data, columns=['Heading', 'SQL Query'])

    return df

# Đọc file Word và trích xuất SQL code vào DataFrame
df_sql = extract_sql_code('JOB nghiệp vụ 2.11.docx')

# Hiển thị DataFrame
df_sql


,Heading,SQL Query
0,Quy tắc lấy dữ liệu,"SELECT \nsm.MARK_USER mark_user,\nsm.COMPANY_A..."
1,Bước 1 : Lấy dữ liệu kỳ đánh giá thỏa mãn điều...,SELECT \n*\nFROM\nTBL_PERIOD_EVALUATION pe\nJO...
2,B2.1 Lấy dữ liệu kỳ trước :,SELECT \n*\nFROM\nTBL_PERIOD_EVALUATION pe\nJO...
3,B2.2 Truy vấn thông tin ngày quá hạn (NO_DAY_O...,SELECT * TBL_CONDITION_CREDIT_RELATION cr WHE...
4,Lấy dữ liệu thỏa mãn điều kiện đánh giá,SELECT \n*\nFROM\nTBL_PERIOD_EVALUATION pe\nJO...
...,...,...
115,Xử lý dữ liệu,SELECT * FROM TBL_PERIOD_EVALUATION WHERE CONT...
116,Lấy dữ liệu thỏa mãn điều kiện,SELECT * FROM TBL_ADVANCE_PAYMENT_RELEASE WHER...
117,Xử lý dữ liệu,SELECT * FROM TBL_BI_GUARANTEE_SALES WHERE CON...
118,Xử lý dữ liệu,SELECT \n*\nFROM\nTBL_CONTRACT_CONDITION\nWHER...


In [11]:
output_file = 'heading_sql_queries.xlsx'
df_sql.to_excel(output_file, index=False)

print(f"File Excel đã được lưu tại: {output_file}")

File Excel đã được lưu tại: heading_sql_queries.xlsx
